In [2]:
from tensorflow.keras.layers import Input, Conv1D, Concatenate, MaxPooling1D, Flatten, Dense, GlobalMaxPooling1D, subtract, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

# baseline model

input_shape = (256, 1) # assuming your signals have length 256, 1 channel

# conv base model
sig_input = Input(input_shape)
cnn1 = Conv1D(64,3,activation='relu',input_shape=input_shape, kernel_regularizer=l2(2e-4))(sig_input)
mp1 = MaxPooling1D()(cnn1)
mp1 = BatchNormalization()(mp1)
cnn2 = Conv1D(128,3,activation='relu', kernel_regularizer=l2(2e-4))(mp1)
mp2 = MaxPooling1D()(cnn2)
mp2 = BatchNormalization()(mp2)
cnn3 = Conv1D(128,3,activation='relu', kernel_regularizer=l2(2e-4))(mp2)
mp3 = MaxPooling1D()(cnn3)
mp3 = BatchNormalization()(mp3)
cnn4 = Conv1D(256,3,activation='relu', kernel_regularizer=l2(2e-4))(mp3)
mp4 = MaxPooling1D()(cnn4)
mp4 = BatchNormalization()(mp4)
flat = Flatten()(mp4)
embed = Dense(64, activation="sigmoid")(flat)

conv_base = Model(sig_input, embed)

conv_base.summary()

left_input = Input(input_shape)
right_input = Input(input_shape)

# encode each of the two inputs into a vector with the base conv model
encoded_l = conv_base(left_input)
encoded_r = conv_base(right_input)



fusion = Concatenate()([encoded_l,encoded_r]) # this can be any other fusion method too

prediction = Dense(1, activation='sigmoid')(fusion)

twin_net = Model([left_input,right_input],prediction)

optimizer = Adam(0.001)

twin_net.compile(loss="binary_crossentropy",optimizer=optimizer)

twin_net.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 256, 1)]          0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 254, 64)           256       
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 127, 64)           0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 127, 64)           256       
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 125, 128)          24704     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 62, 128)           0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 62, 128)           512 